In [1]:
from __future__ import division, print_function, absolute_import

import numpy as np
import tflearn
import tensorflow as tf
from datetime import datetime
from sklearn.metrics import mean_squared_error
import math

In [2]:
import progressbar

In [3]:
from combine import X, Y

In [4]:
Y = np.array(Y).reshape((len(Y),1)) # KEY

In [5]:
tf.reset_default_graph()

In [6]:
run_identification = 'econ-' + str(datetime.now())[:-7]
# Network building
net = tflearn.input_data([None, 3]) # input layer which receives any number of input data (evidenced by None), each containing 3 nodes 
# first fully connected layer containing 10 neurons, a linear activation function (keep whatever you outputted), L2 regularization (prevent overfitting), and weight decay
net = tflearn.fully_connected(net, 10, activation='linear',
                                 regularizer='L2', weight_decay=0.0005)
# last layer which contains only one output neuron with linear activation
net = tflearn.fully_connected(net, 1, activation='linear')
# regression layer using AdaGrad to optimize, learning rate of 0.01, mean-square loss
net = tflearn.regression(net, optimizer=
                         tflearn.optimizers.AdaGrad(learning_rate=0.01, initial_accumulator_value=0.01), 
                         loss='mean_square', learning_rate=0.05)

# Training
# Deep Neural Network Model created
model = tflearn.DNN(net, tensorboard_verbose=3, checkpoint_path='tmp/',
                   tensorboard_dir="/tmp/tflearn_logs/")
# Fit the model using X and Y as inputs and outputs, 1000 epochs, showing metrics, shuffling data, and with a batch size of 100
model.fit(X, Y, n_epoch=1000, show_metric=True,
          run_id=run_identification,
          shuffle=False,
          batch_size=100)

Training Step: 6999  | total loss: 5.89213 | time: 0.121s
| AdaGrad | epoch: 1000 | loss: 5.89213 - binary_acc: 0.0088 -- iter: 600/687
Training Step: 7000  | total loss: 5.58785 | time: 0.140s
| AdaGrad | epoch: 1000 | loss: 5.58785 - binary_acc: 0.0079 -- iter: 687/687
--
INFO:tensorflow:/Users/rodrigo.castellon/Desktop/econ-fed/tmp-7000 is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
def gen_y_pred(model, X):
    y_pred = np.array(model.predict(X)).squeeze()
    return y_pred

In [8]:
def eval_model(y_pred, y_actual): # Returns: average abs deviation, average deviation, 
    mean_squared = find_deviation(y_pred, y_actual, method='mse')
    
    return mean_squared

In [9]:
def find_deviation(y_pred, y_actual, method):
    deviation_sum = 0
    if method == 'absolute':
        for i in range(len(Y)):
            deviation_sum += np.abs(model.predict([X[i]])[0][0] - Y[i][0])
    elif method == 'mse':
        deviation = math.sqrt(mean_squared_error(y_actual, y_pred))
    else:
        pass
    return deviation

In [10]:
eval_model(gen_y_pred(model, X), Y.squeeze())

2.4727233889349884

In [20]:
Y.shape

(687, 1)